# Imports 

In [10]:
#export
import os
import datetime
import collections
from pathlib import Path
from collections import OrderedDict

import numpy as np
import pandas as pd

# Code 

In [40]:
#export
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def load_txt_log(path, types=None, read_header=False, delimiter=','):
    with open(path, 'r') as f:
        raw_logs = f.read()
    logs = raw_logs.split('\n')

    header=None
    if read_header:
        header, *logs = logs
        header = header.split(delimeter)

    parsed_logs = []
    for line in logs:
        line = line.split(delimiter)
        if types is not None:
            line = [t(l) for t,l in zip(types,line)]
        parsed_logs.append(line)

    return header, parsed_logs

class BaseProject:
    def __init__(self, root, valid_func, Experiment):
        self.root = root
        self.exps = []
        for p in root.iterdir():  
            if valid_func(p):
                self.exps.append(Experiment(p))
        
    def load(self):
        raise NotImplementedError
    
    def base_table(self):
        run_dfs = []
        for e in self.exps:
            dfi = pd.DataFrame(e.cfg_data, index=[e.name])
            run_dfs.append(dfi)
        df = pd.concat(run_dfs)
        return df
    
class BaseExperiment:
    def __init__(self, path, log_reader, cfg_reader):
        self.path = path
        self.name = path.name
        self.log_reader = log_reader
        self.cfg_reader = cfg_reader
        self.log_data, self.cfg_data = self.load()
    
    
    def load(self):
        log_data = self.log_reader(self.path)
        cfg_data = self.cfg_reader(self.path)
        return log_data, cfg_data
    
    def __repr__(self):
        return str(__class__) + self.name
    
    def __len__(self):
        return len(self.log_data)
        
    def best(self, col, num=1, ascending=False):
        assert col in self.log_data.columns
        return self.log_data.sort_values(col, ascending=ascending)[:num]

# Exports

In [1]:
!python3 extra/n2s.py base_exp.ipynb

Converted base_exp.ipynb to exp/nb_base_exp.py
